In [ ]:
import dotenv
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import dotenv
from tqdm.auto import tqdm

In [ ]:
dotenv.load_dotenv(dotenv.find_dotenv())

[lien](https://apisolidarite.soliguide.fr/Documentation-technique-de-l-API-Solidarit-ecaf8198f0e9400d93140b8043c9f2ce) vers la documentation api soliguide

In [ ]:
pd.options.display.max_rows = None
plt.rc("figure", figsize=[12, 8])
tqdm.pandas()

### chargement des données soliguide

In [ ]:
soliguide_raw_df = pd.read_json("soliguide-places-20220721.json")
soliguide_raw_df = soliguide_raw_df.replace(["", np.nan], None)

In [ ]:
soliguide_services_df = pd.json_normalize(
    soliguide_raw_df.to_dict(orient="records"),
    record_path="services_all",
    meta="lieu_id",
)

# 2. Trouver les SIAE correspondants à ces SIRET

In [ ]:
# Récupération des siaes en passant les services catégorisés avec le code 205 ("Insertion par l'activité économique")
soliguide_siaes_df = soliguide_raw_df[
    soliguide_raw_df.lieu_id.isin(
        soliguide_services_df[soliguide_services_df.categorie == 205].lieu_id.to_list()
    )
]

soliguide_siaes_df.shape[0]


In [ ]:
from data_inclusion.scripts.tasks.sources import soliguide

soliguide_siaes_df = soliguide.transform_dataframe(soliguide_siaes_df)

In [ ]:
from data_inclusion.scripts.tasks import geocoding

soliguide_siaes_df = geocoding.geocode_normalized_dataframe(
    soliguide_siaes_df,
    geocoding.BaseAdresseNationaleBackend(base_url="https://api-adresse.data.gouv.fr/"),
)


# 1. Trouver les SIRET automatique pour les données Soliguide

In [ ]:
from data_inclusion.scripts.tasks.siretisation import siretisation

soliguide_siaes_df = siretisation.siretize_normalized_dataframe(
    soliguide_siaes_df,
    os.environ["SIRENE_DATABASE_URL"],
)


In [ ]:
soliguide_siaes_df.siret.notna().sum()

8 erreurs

In [ ]:
# exclude errors
soliguide_siaes_df= soliguide_siaes_df[
    ~soliguide_siaes_df.siret.isin(
        [
            "49929393400018",
            "79493735900014",
            "49825896100024",
            "82068992500015",
            "39900733500017",
            "33958232200014",
            "85154273800014",
            "90840665500013",
        ]
    )
]


### chargement des données siaes des emplois de l'inclusion

In [ ]:
from data_inclusion.scripts.tasks.sources import itou

client = itou.ItouClient(
    url="https://emplois.inclusion.beta.gouv.fr/api/v1/structures/?type=siae",
    token=os.environ["ITOU_API_TOKEN"],
)

itou_siaes_df = pd.DataFrame(client.list_structures()).replace([np.nan, ""], None)

In [ ]:
from data_inclusion.scripts.tasks import geocoding

itou_siaes_df = geocoding.geocode_normalized_dataframe(
    itou_siaes_df,
    geocoding.BaseAdresseNationaleBackend(base_url="https://api-adresse.data.gouv.fr/"),
)


In [ ]:
itou_siaes_df.shape[0]

# 3. croisement emplois de l'inclusion / soliguide sur la base des sirets retrouvés automatiquement

In [ ]:
shared_df = pd.merge(
    soliguide_siaes_df.add_prefix("soliguide_"),
    itou_siaes_df.drop_duplicates(["siret"]).add_prefix("itou_"),
    left_on="soliguide_siret",
    right_on="itou_siret",
)

shared_df.shape[0]

In [ ]:
shared_df[["soliguide_siret", "soliguide_id", "itou_id"]]

In [ ]:
shared_df[["soliguide_siret", "soliguide_date_maj", "itou_date_maj"]].assign(
    latest=shared_df.apply(
        lambda row: "soliguide"
        if row["soliguide_date_maj"] > row["itou_date_maj"]
        else "itou",
        axis=1,
    )
)


Departements communs ?

In [ ]:
shared_df.soliguide_code_insee.str[:2].value_counts().to_frame()

# 4. Trouver tous les SIAE dans les emplois sur les territoires de Soliguide

* Territoires de Soliguide : départements où soliguide a au moins 50 structures
* on exclut les SIAEs déjà dans les données de Soliguide

In [ ]:
from data_inclusion.scripts.tasks.sources import soliguide
from data_inclusion.scripts.tasks import geocoding

soliguide_structures_df = soliguide.transform_dataframe(soliguide_raw_df)
soliguide_structures_df = geocoding.geocode_normalized_dataframe(
    soliguide_structures_df,
    geocoding.BaseAdresseNationaleBackend(base_url="https://api-adresse.data.gouv.fr/"),
)

In [ ]:
count_by_cog_dept = soliguide_structures_df.code_insee.str[:2].value_counts()

nb de siaes des emplois de l'inclusion sur les territoires de soliguide ?

In [ ]:
itou_siaes_of_interest_df = itou_siaes_df[itou_siaes_df.code_insee.str[:2].isin(count_by_cog_dept[count_by_cog_dept > 50].index.to_list())]
itou_siaes_of_interest_df.shape[0]

nb de siaes des emplois de l'inclusion inconnues par soliguide sur ses territoires ?

In [ ]:
itou_siaes_of_interest_df = itou_siaes_of_interest_df[~itou_siaes_of_interest_df.siret.isin(soliguide_siaes_df.siret)]
itou_siaes_of_interest_df.shape[0]

In [ ]:
soliguide_siaes_df.shape[0]